In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import intervaltree


In [2]:
bases = dict(zip("ACGT", [0,1,2,3]))
bases


{'A': 0, 'C': 1, 'G': 2, 'T': 3}

In [3]:
def get_confidence_regions(file):
    confidence_regions = intervaltree.IntervalTree()
    with open(file) as f:
        for row in f:
            confidence_regions.addi(row[1], row[2])
            
    return confidence_regions


In [ ]:
# [0,0,0,0,0,0,0,0] first 4 with bases, then if it is hetero
# homo, non-variant or complex variant
def create_tensors(variants, high_confidence_region, aln_tensor):
    
    confidence_regions = get_confidence_regions(high_confidence_region)
    Ys = {}
    
    # read the variants
    with open(variants) as variants:
        for row in variants:
            row = row.strip().split()
            tensor = dict(zip([0,0,0,0,0,0,0,0],
            ["A","C","G","T","het", "homo", "non_variant", "complex_variant"]))
            
            position = int(row[0])
            # not in confidence regions
            if len(confidence_regions.search(position)) == 0:
                continue
            
            # not a simple SNP
            elif len(row[1]) > 1 or len(row[2]) > 1:
                tensor["complex_variant"] = 1
                
            # hetero
            elif row[3] == 0:
                tensor["het"] = 1
                tensor[row[1][0]] = 0.5
                tensor[row[2][0]] = 0.5
                
            # mono
            else:
                tensor["homo"] = 1
                tensor[row[2][0]] = 1
            
            Ys[position] = tensor.values()
        
    Y_positions = sorted(Ys.keys())
    
    for index, item in enumerate(Y_positions):
        previous_item = Y_positions[index - 1]
        if abs(item - previous_item) < 12:
            Ys[item] = [0,0,0,0,0,0,0,1]
            Ys[previous_item] = [0,0,0,0,0,0,0,1]
            
    with open(aln_tensor) as file: 
        for row in file:
            row = row.strip().split()
            position = int(row[0])
            ref
            
            # not in confidence regions
            if len(confidence_regions.search(position)) == 0:
                continue

                
            
            